In [1]:
import numpy as np

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [3]:
import seaborn as sns


In [4]:
df=sns.load_dataset('tips')

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
X=df.iloc[:,1:]
y=df['total_bill']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


In [9]:
numeric_preprocessor = Pipeline( steps=[ ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")), ("scaler", StandardScaler()), ])

In [10]:
from sklearn import set_config

In [11]:
numeric_preprocessor

,steps,"[('imputation_mean', ...), ('scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [12]:
categorical_preprocessor = Pipeline( steps=[ ( "imputation_constant", SimpleImputer(fill_value="missing", strategy="constant"), ),("onehot", OneHotEncoder(handle_unknown="ignore")), ])
preprocessor=Pipeline( steps=[("categorical",categorical_preprocessor),("numerical",numeric_preprocessor)])
preprocessor

,steps,"[('categorical', ...), ('numerical', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('imputation_constant', ...), ('onehot', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'constant'
,fill_value,'missing'


In [14]:
Pipeline(steps=[('categorical', Pipeline(steps=[('imputation_constant', SimpleImputer(fill_value='missing', strategy='constant')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])), ('numerical', Pipeline(steps=[('imputation_mean', SimpleImputer()), ('scaler', StandardScaler())]))])

,steps,"[('categorical', ...), ('numerical', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('imputation_constant', ...), ('onehot', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'constant'
,fill_value,'missing'


In [15]:
pipe=Pipeline( [("preprocessor",preprocessor),("regressor",RandomForestRegressor())])

In [16]:
pipe

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('categorical', ...), ('numerical', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('imputation_constant', ...), ('onehot', ...)]"
,transform_input,None
,memory,None


In [17]:
Pipeline(steps=[('preprocessor', Pipeline(steps=[('categorical', Pipeline(steps=[('imputation_constant', SimpleImputer(fill_value='missing', strategy='constant')),('onehot', OneHotEncoder(handle_unknown='ignore'))])), ('numerical', Pipeline(steps=[('imputation_mean', SimpleImputer()), ('scaler', StandardScaler())]))])), ('regressor', RandomForestRegressor())])

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('categorical', ...), ('numerical', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('imputation_constant', ...), ('onehot', ...)]"
,transform_input,None
,memory,None


In [19]:
from sklearn.compose import ColumnTransformer

# Define categorical and numerical columns
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Correct: Use ColumnTransformer to apply preprocessors column-wise
preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", categorical_preprocessor, categorical_cols),
        ("numerical", numeric_preprocessor, numerical_cols),
    ]
)

pipe = Pipeline(
    [("preprocessor", preprocessor), ("regressor", RandomForestRegressor())]
)

pipe.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('categorical', ...), ('numerical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
